In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
#Load the trained ANN Model, scaler,ohe
model=load_model('model.keras')

with open('ohe_geography.pkl','rb') as file:
    ohe_geography=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_gen=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [3]:
#Example Input
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [4]:
#OHE on Geogrpahy
geo_encoded = ohe_geography.transform(
    pd.DataFrame({'Geography': [input_data['Geography']]})
)

geoencoded_df = pd.DataFrame(
    geo_encoded,
    columns=ohe_geography.get_feature_names_out()
)


In [5]:
input_df=pd.DataFrame([input_data])
input_df=input_df.drop(columns=['Geography'],axis=1)

In [6]:
input_df=pd.concat([input_df,geoencoded_df],axis=1)

In [7]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
#Encode gender
input_df['Gender']=label_gen.transform(input_df['Gender'])

In [9]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
#Apply scaling
encoded_data=scaler.transform(input_df)

In [11]:
encoded_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

Prediction

In [12]:
prediction=model.predict(encoded_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


In [13]:
prediction

array([[0.03862688]], dtype=float32)

In [14]:
prediction_proba=prediction[0][0]

In [15]:
prediction_proba

np.float32(0.038626883)

In [16]:
if prediction_proba>0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
